In [1]:
import sys
sys.path.append("..")
import numpy as np
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time
from pathlib import Path
from datetime import datetime

In [2]:
# Класс для сохранения результатов
tests = Tests('test1')
Path("../tests/test1/d20").mkdir(parents=True, exist_ok=True)
print("Время последнего моделирования:", datetime.now().strftime("%d/%m/%Y %H:%M"))

# Создаем наблюдателя
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

Время последнего моделирования: 22/03/2021 21:49


In [3]:
# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [4]:
def target_func(seed=None):
    np.random.seed(seed)
    b = 0
    d = 20
    c = np.random.uniform(0, 180)
    v = np.random.uniform(3, 15)
    return [b, d, c, v]
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
precisions_1 = ['{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}',
'{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}','{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{:.0f}', '{0:.4f}', '{0:.0f}', '{0:.0f}']
precisions_2 = ['{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}', '{0:.2f}', '{0:.2f}', '{0:.0f}', '{0:.4f}']

In [5]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    dict_results = tma.swarm(n=100, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results)
    tma.print_verbose()

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('Nf = {:.0f}, Nf_max = {:.0f}'.format(df['Nf'].mean(), max(df['Nf'])))
    print('Iter = {:.0f}, Iter_max = {:.0f}'.format(df['Iter'].mean(), max(df['Iter'])))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['СКО X', 'СКО Y', 'СКО VX', 'СКО VY', 'Кб', 'П0_апр', 'Д0_апр', 'К0_апр', 'V0_апр'], axis=1).head())

    tests.save_df(df, name='../tests/test1/d20/ЛМ, ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_________________________________________________')

Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.9 с
СКОп = 0.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0032, t_max = 0.0115


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,20.000,98.786,11.58,0.0,20.000,98.786,11.58,44.33,21.333,44.33,21.332,0.00,"[1, 1, 1, 1]",0.0115,10,9
1,0.0,20.000,75.064,11.64,0.0,20.000,75.064,11.64,34.96,25.605,34.96,25.605,0.00,"[1, 1, 1, 1]",0.0048,10,9
2,0.0,20.000,78.479,3.31,360.0,20.000,78.479,3.31,15.57,18.869,15.57,18.869,0.00,"[1, 1, 1, 1]",0.0027,8,7
3,0.0,20.000,99.144,11.50,0.0,20.000,99.144,11.50,44.24,21.203,44.24,21.203,0.00,"[1, 1, 1, 1]",0.0029,10,9
4,0.0,20.000,174.065,9.57,360.0,20.000,174.065,9.57,21.56,6.414,21.56,6.414,0.00,"[1, 1, 1, 1]",0.0026,9,8


_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.8 с
СКОп = 0.1, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.01
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0028, t_max = 0.0063


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,20.000,98.786,11.58,360.0,20.062,98.701,11.65,44.33,21.333,44.35,21.439,0.01,"[1, 1, 1, 1]",0.0028,10,9
1,0.0,20.000,75.064,11.64,0.0,19.908,75.064,11.56,34.96,25.605,34.94,25.456,0.01,"[1, 1, 1, 1]",0.0028,10,9
2,0.0,20.000,78.479,3.31,0.0,20.046,79.766,3.29,15.57,18.869,15.58,18.823,0.01,"[1, 1, 1, 1]",0.0022,8,7
3,0.0,20.000,99.144,11.50,0.0,19.906,99.296,11.40,44.24,21.203,44.23,21.045,0.01,"[1, 1, 1, 1]",0.0026,10,9
4,0.0,20.000,174.065,9.57,0.0,19.995,174.132,9.58,21.56,6.414,21.55,6.388,0.01,"[1, 1, 1, 1]",0.0024,9,8


_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.8 с
СКОп = 0.2, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 98%, Рэф_мин = 0
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.04
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0028, t_max = 0.0109


...

_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.8 с
СКОп = 0.5, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 79%, Рэф_мин = 0
Вариант 1: Рэф = 97%, Рэф_мин = 0
Вариант 2: Рэф = 97%, Рэф_мин = 0
Вариант 3: Рэф = 98%, Рэф_мин = 0
Ка = 0.25
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0027, t_max = 0.0061


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,20.000,98.786,11.58,359.9,20.317,98.369,11.95,44.33,21.333,44.45,21.871,0.25,"[1, 1, 1, 1]",0.0030,10,9
1,0.0,20.000,75.064,11.64,0.0,19.551,75.064,11.25,34.96,25.605,34.86,24.879,0.25,"[1, 1, 1, 1]",0.0028,10,9
2,0.0,20.000,78.479,3.31,0.1,20.231,85.105,3.21,15.57,18.869,15.65,18.634,0.26,"[0, 1, 1, 1]",0.0022,8,7
3,0.0,20.000,99.144,11.50,0.1,19.541,99.916,11.02,44.24,21.203,44.19,20.426,0.26,"[1, 1, 1, 1]",0.0027,10,9
4,0.0,20.000,174.065,9.57,0.0,19.976,174.394,9.64,21.56,6.414,21.48,6.284,0.24,"[1, 1, 1, 1]",0.0027,9,8


_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.8 с
СКОп = 1.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 44%, Рэф_мин = 0
Вариант 1: Рэф = 80%, Рэф_мин = 0
Вариант 2: Рэф = 81%, Рэф_мин = 0
Вариант 3: Рэф = 92%, Рэф_мин = 0
Ка = 0.99
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0027, t_max = 0.0058


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,20.000,98.786,11.58,359.8,20.645,97.971,12.33,44.33,21.333,44.57,22.426,1.00,"[0, 1, 1, 1]",0.0026,10,9
1,0.0,20.000,75.064,11.64,0.0,19.126,75.065,10.88,34.96,25.605,34.76,24.192,0.99,"[0, 1, 1, 1]",0.0024,9,8
2,0.0,20.000,78.479,3.31,0.1,20.460,92.130,3.14,15.57,18.869,15.73,18.392,1.05,"[0, 0, 0, 0]",0.0021,8,7
3,0.0,20.000,99.144,11.50,0.2,19.106,100.717,10.57,44.24,21.203,44.13,19.689,1.04,"[0, 1, 1, 1]",0.0026,10,9
4,0.0,20.000,174.065,9.57,0.0,19.955,174.709,9.71,21.56,6.414,21.40,6.156,0.94,"[1, 1, 1, 1]",0.0023,9,8


_________________________________________________


In [6]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    algorithm_name = 'Метод N пеленгов'
    dict_results = tma.swarm(n=100, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results).dropna(axis=1)
    tma.print_verbose()

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))
    
    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['Кб'], axis=1).head())

    tests.save_df(df, name='../tests/test1/d20/N, ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_________________________________________________')

Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.6 с
СКОп = 0.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
t = 0.0004, t_max = 0.0031


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,20.000,98.786,11.58,0.0,20.000,98.786,11.58,44.3,21.333,44.327,21.33,0.00,"[1, 1, 1, 1]",0.001
1,0.0,20.000,75.064,11.64,0.0,20.000,75.064,11.64,35.0,25.605,34.958,25.60,0.00,"[1, 1, 1, 1]",0.002
2,0.0,20.000,78.479,3.31,0.0,20.000,78.479,3.31,15.6,18.869,15.567,18.87,0.00,"[1, 1, 1, 1]",0.000
3,0.0,20.000,99.144,11.50,0.0,20.000,99.144,11.50,44.2,21.203,44.238,21.20,0.00,"[1, 1, 1, 1]",0.000
4,0.0,20.000,174.065,9.57,0.0,20.000,174.065,9.57,21.6,6.414,21.564,6.41,0.00,"[1, 1, 1, 1]",0.000


_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 0.1, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 76%, Рэф_мин = 0
Вариант 1: Рэф = 95%, Рэф_мин = 0
Вариант 2: Рэф = 95%, Рэф_мин = 0
Вариант 3: Рэф = 99%, Рэф_мин = 0
Ка = 0.01
t = 0.0003, t_max = 0.0007


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,20.000,98.786,11.58,359.8,20.298,97.853,11.99,44.3,21.333,44.358,21.98,0.01,"[1, 1, 1, 1]",0.000
1,0.0,20.000,75.064,11.64,359.8,20.017,74.176,11.93,35.0,25.605,34.942,25.99,0.01,"[1, 1, 1, 1]",0.000
2,0.0,20.000,78.479,3.31,0.0,19.971,81.630,3.20,15.6,18.869,15.557,18.60,0.01,"[1, 1, 1, 1]",0.000
3,0.0,20.000,99.144,11.50,359.8,20.229,98.149,11.85,44.2,21.203,44.240,21.77,0.01,"[1, 1, 1, 1]",0.000
4,0.0,20.000,174.065,9.57,360.0,19.960,174.131,9.56,21.6,6.414,21.530,6.38,0.01,"[1, 1, 1, 1]",0.000


_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 0.2, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 57%, Рэф_мин = 0
Вариант 1: Рэф = 74%, Рэф_мин = 0
Вариант 2: Рэф = 75%, Рэф_мин = 0
Вариант 3: Рэф = 88%, Рэф_мин = 0
Ка = 0.04
t = 0.0003, t_max = 0.0010


...

_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 0.5, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 0%, Рэф_мин = 0
Вариант 1: Рэф = 11%, Рэф_мин = 0
Вариант 2: Рэф = 18%, Рэф_мин = 0
Вариант 3: Рэф = 36%, Рэф_мин = 0
Ка = 0.32
t = 0.0003, t_max = 0.0022


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,20.000,98.786,11.58,359.1,17.690,93.987,10.67,44.3,21.333,43.951,19.70,0.38,"[0, 1, 1, 1]",0.000
1,0.0,20.000,75.064,11.64,359.2,16.778,69.926,10.36,35.0,25.605,34.389,22.33,0.39,"[0, 0, 0, 1]",0.000
2,0.0,20.000,78.479,3.31,0.2,18.446,102.336,2.45,15.6,18.869,15.139,15.75,0.30,"[0, 0, 0, 0]",0.000
3,0.0,20.000,99.144,11.50,359.1,17.076,93.936,10.10,44.2,21.203,43.642,18.83,0.46,"[0, 0, 0, 1]",0.001
4,0.0,20.000,174.065,9.57,360.0,19.028,175.258,9.18,21.6,6.414,20.907,5.81,0.26,"[0, 1, 1, 1]",0.000


_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 1.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 0%, Рэф_мин = 0
Вариант 1: Рэф = 0%, Рэф_мин = 0
Вариант 2: Рэф = 0%, Рэф_мин = 0
Вариант 3: Рэф = 0%, Рэф_мин = 0
Ка = 2.00
t = 0.0003, t_max = 0.0006


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,20.000,98.786,11.58,358.2,10.399,86.330,5.38,44.3,21.333,41.679,10.97,3.19,"[0, 0, 0, 0]",0.000
1,0.0,20.000,75.064,11.64,358.4,10.385,61.005,5.94,35.0,25.605,32.357,13.28,2.77,"[0, 0, 0, 0]",0.000
2,0.0,20.000,78.479,3.31,0.4,15.041,149.526,2.01,15.6,18.869,13.581,10.65,1.62,"[0, 0, 0, 0]",0.001
3,0.0,20.000,99.144,11.50,358.2,9.793,85.167,4.84,44.2,21.203,40.964,10.14,3.81,"[0, 0, 0, 0]",0.000
4,0.0,20.000,174.065,9.57,359.9,16.678,178.392,8.21,21.6,6.414,18.762,4.45,1.43,"[0, 0, 0, 0]",0.000


_________________________________________________
